In [23]:
"""Define the loss function between the y_train_estimated and y_train
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
"""
def criterion(y_estimated, y):

    # http://pytorch.org/docs/master/nn.html : torch.nn.NLLLoss
    nllcrit = nn.NLLLoss2d(reduce = True)

    # Apply softmax then the log on the result
    y_estimated = F.log_softmax(y_estimated, dim = 1)

    # (y!= parameters.number_classes) is a matrix with 0 on position were the target is class parameters.number_classes
    # unsqueeze ad a dimension to allowed multiplication and float transform the Variable to a float Variable
    y_estimated = y_estimated * (y != parameters.number_classes ).unsqueeze(1).float()
    
    # Set all target value of number_classes to 0. The nllcrit will do y_estimated[k,0,i,j]*y[k,i,j]
    # It will be 0 if the class is parameters.number_classes : which is exactly what isexpect for this class
    # The other classes remain unchanged
    y[y == parameters.number_classes] = 0

    # Apply the criterion define in the first line
    return nllcrit(y_estimated, y)

In [16]:
"""Define the loss function between the y_estimated and y and return a vector with the epoch the value
    and if the criterion is used on training or validation set.
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
    (2) = epoch : the actual epoch of the learning
    (3) = set_type : string which is train of validation
"""
def criterion_pd_format(y_estimated, y, epoch, set_type):
    # http://pytorch.org/docs/master/nn.html : torch.nn.NLLLoss
    nllcrit = nn.NLLLoss2d(reduce = True)

    # Apply softmax then the log on the result
    y_estimated = F.log_softmax(y_estimated, dim = 1)

    # (y!= parameters.number_classes) is a matrix with 0 on position were the target is class parameters.number_classes
    # unsqueeze ad a dimension to allowed multiplication and float transform the Variable to a float Variable
    y_estimated = y_estimated * (y != parameters.number_classes ).unsqueeze(1).float()
    
    # Set all target value of number_classes to 0. The nllcrit will do y_estimated[k,0,i,j]*y[k,i,j]
    # It will be 0 if the class is parameters.number_classes : which is exactly what isexpect for this class
    # The other classes remain unchanged
    y[y == parameters.number_classes] = 0
    
    # Apply the criterion define in the first line
    loss = nllcrit(y_estimated, y)
    
    # Return a vector  usefull to copy to CSV 
    return ([loss.data[0], set_type, epoch])

In [20]:
"""
    Test function : IoU
    Return a matrix of confusion in a good format to creat a pandas DataFrame
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
    (2) = epoch : the actual epoch of the learning
    (3) = set_type : string which is train of validation
"""
def IoU_pd_format(y_estimated, y, set_type, epoch):
    
    #We keep only the higest value, which is the prediction
    pred = torch.max(y_estimated, dim=1)[1]

    #Creat the confusion matrix, only the second column will have the value
    confusion_matrix = [[0] * 5 for i in range(parameters.number_classes**2)]
    
    pred = pred.view(-1)
    target = y.view(-1)

    # We will normalise the value in the matrix. We don t want to be influence by the size of the batch
    normalisation_value = y.size()

    # It is also possible to add the size of the image to normalise
    normalisation_value = normalisation_value[0]# * normalisation_value[1] * normalisation_value[2]
    
    # Double loop over the number of classes at each iteration we add the intersection
    # i will be the number of iteration
    i = 0
    for cls1 in range(parameters.number_classes):
        pred_inds = pred == cls1
        
        for cls2 in range(parameters.number_classes):
            
            i = cls1*parameters.number_classes + cls2
            
            target_inds = target == cls2
            # Intersection is the value predicted of class cls2 and are in reality class cls1
            intersection = (pred_inds*target_inds).long().sum().data.cpu()[0]
            
            # Traning or validation set
            confusion_matrix[i][0] = set_type
            # The value normalised
            confusion_matrix[i][1] = intersection/normalisation_value
            # Associated with this value we keep the two classes
            confusion_matrix[i][2] = "class" + str(cls1)
            confusion_matrix[i][3] = "class" + str(cls2)
            # The epoch associated
            confusion_matrix[i][4] = epoch

    return(confusion_matrix)
    